In [4]:
from archivum.db import Archivum
import archivum.config as config

db = Archivum(
    storage_path=config.VECTOR_DB_PATH,
    collection_name=config.COLLECTION_NAME
)


[Embedder|Load] Loading embedder model: intfloat/multilingual-e5-large-instruct on device: cpu
[Embedder|Load] Model loaded successfully.
[Archivum|Init] Connected to collection 'archivum_fragments' at '/Users/soho/Archivum Mentis/code/mentis-local/archivum/vectordb'.


In [5]:
from archivum.ingest import ingest
ingest()


[Tokenizer|Load] Loading tokenizer from: assets/tokenizer.json
[Tokenizer|Load] Tokenizer loaded. Padding to 512 tokens.
[Embedder|Load] Loading embedder model: intfloat/multilingual-e5-large-instruct on device: cpu
[Embedder|Load] Model loaded successfully.
[Archivum|Init] Connected to collection 'archivum_fragments' at '/Users/soho/Archivum Mentis/code/mentis-local/archivum/vectordb'.
[Ingest] New file detected: /Users/soho/Archivum Mentis/code/mentis-local/input/sample/treaty_of_versaille_pdf.pdf
[Parser|Dispatch] File extension detected: .pdf
[Parser|PDF] Parsing file: /Users/soho/Archivum Mentis/code/mentis-local/input/sample/treaty_of_versaille_pdf.pdf
[Parser|PDF] Parsed 10 pages with text.
[Ingest] New file detected: /Users/soho/Archivum Mentis/code/mentis-local/input/sample/cactus.md
[Parser|Dispatch] File extension detected: .md
[Parser|MD] Parsing file: /Users/soho/Archivum Mentis/code/mentis-local/input/sample/cactus.md
[Parser|MD] Parsed markdown file into one chunk.
[Inge

In [6]:
import json
from archivum.query import retrieve_relevant_chunks, retrieve_chunks_with_metadata
query_text = input("Enter your query: ")
chunks = retrieve_chunks_with_metadata(query_text, n_results=5)
print("\n=== Retrieved Chunks ===")
for idx, chunk in enumerate(chunks, 1):
    print(f"Chunk {idx}:")
    print(f"Text: {chunk['text']}")
    print(f"Metadata: {json.dumps(chunk['metadata'], indent=2)}")
    print(f"Distance: {chunk['distance']}")
    print("-" * 40)

[Embedder|Load] Loading embedder model: intfloat/multilingual-e5-large-instruct on device: cpu
[Embedder|Load] Model loaded successfully.
[Archivum|Init] Connected to collection 'archivum_fragments' at '/Users/soho/Archivum Mentis/code/mentis-local/archivum/vectordb'.
[Query|Retrieve+Meta] Retrieving top 5 chunks + metadata for query: 'What are the tradeoffs between learned positional embeddings and sinusoidal ones?'
[Archivum|Query] Querying for: 'What are the tradeoffs between learned positional embeddings and sinusoidal ones?' | Top 5 results
[Embedder|Embed] Embedding 1 texts... Batch size: 32 | Normalize: True | Tensor Output: False
[Archivum|Query] Retrieved 5 chunks.
[Query|Retrieve+Meta] Retrieved 5 chunks with metadata.

=== Retrieved Chunks ===
Chunk 1:
Text: In row (E) we replace our
sinusoidal positional encoding with learned positional embeddings [9], and observe nearly identical
results to the base model.
Metadata: {
  "page_number": 9,
  "file_path": "/Users/soho/Archivu